# ClyptQ Quickstart

Quick start guide:
1. Configure DataSpec & Dynamic Universe
2. Load data with DataProvider
3. Calculate signals with operators
4. Define Strategy
5. Run Engine backtest

In [ ]:
# Auto-reload for development
%load_ext autoreload
%autoreload 2

from datetime import datetime
import pandas as pd
import numpy as np

# ClyptQ imports
from clyptq import operator
from clyptq.data.provider import DataProvider
from clyptq.data.spec import OHLCVSpec
from clyptq.universe import CryptoLiquid, StaticUniverse
from clyptq.strategy import Strategy
from clyptq.trading.engine import Engine

print("Imports successful!")

## 1. Configuration

- **OHLCVSpec**: Specify exchange, market_type, timeframe → automatic data path determination
- **Universe**: Define trading symbols (Static or Dynamic)

In [ ]:
# Define DataSpec
ohlcv_spec = OHLCVSpec(
    exchange="gateio",
    market_type="spot",
    timeframe="1d",
)

# Dynamic Universe: Top 30 by volume
universe = CryptoLiquid(
    top_n=30,
    min_dollar_volume=100_000,
)

# Time period
START = datetime(2025, 10, 15)
END = datetime(2026, 1, 5)

print(f"Spec: {ohlcv_spec.exchange}/{ohlcv_spec.market_type}/{ohlcv_spec.timeframe}")
print(f"Universe: {universe.name}")
print(f"Period: {START} ~ {END}")

## 2. Load Data (Research Mode)

In [ ]:
# Create DataProvider
p = DataProvider(
    universe=universe,
    specs={"ohlcv": ohlcv_spec},
    rebalance_freq="1d",
    mode="research",
)

# Load data
p.load(start=START, end=END)

print(f"Total symbols: {len(p.symbols)}")
print(f"Universe symbols: {p.n_universe}")
print(f"Date range: {p['close'].index[0]} ~ {p['close'].index[-1]}")

## 3. Calculate Signals with Operators

Inside Strategy, you must **use pure operators only**.

In [ ]:
close = p["close"]
volume = p["volume"]

# Momentum: 20-day returns
returns_20d = operator.ts_returns(close, period=20)
momentum = operator.rank(returns_20d)  # [0, 1] range

# Mean Reversion: Z-score
ma_20 = operator.ts_mean(close, 20)
std_20 = operator.ts_std(close, 20)
zscore = operator.div(operator.sub(close, ma_20), operator.add(std_20, 1e-8))
mean_rev = operator.rank(operator.neg(zscore))  # Low z-score = high rank

# Combine signals
combined = operator.ca_reduce_avg(momentum, mean_rev)

# Normalize (Spot: long-only)
signal = operator.rank(combined)
signal = operator.l1_norm(signal)  # Sum=1

print(f"Signal shape: {signal.shape}")
print(f"Signal sum (last row): {signal.iloc[-1].sum():.4f}")

## 4. Define Strategy

In [ ]:
class MomentumStrategy(Strategy):
    """Simple Momentum Strategy (Spot Long-Only)."""
    
    name = "Momentum_Spot"
    
    data = {
        "ohlcv": OHLCVSpec(
            exchange="gateio",
            market_type="spot",
            timeframe="1d",
        ),
    }
    
    universe = CryptoLiquid(top_n=30, min_dollar_volume=100_000)
    rebalance_freq = "1d"
    
    def warmup_periods(self) -> int:
        return 30
    
    def compute_signal(self):
        close = self.provider["close"]
        
        # 20-day momentum
        returns_20d = operator.ts_returns(close, period=20)
        signal = operator.rank(returns_20d)
        signal = operator.l1_norm(signal)  # Spot: positive only, sum=1
        
        return signal

print("Strategy defined!")

## 5. Run Backtest

In [ ]:
engine = Engine()

result = engine.run(
    strategy=MomentumStrategy(),
    mode="backtest",
    start=START,
    end=END,
    initial_capital=10000.0,
    market_type="spot",
    verbose=True,
)

print(f"\n=== Results ===")
if result.metrics:
    print(f"Total Return: {result.metrics.total_return:.2%}")
    print(f"Sharpe Ratio: {result.metrics.sharpe_ratio:.2f}")
    print(f"Max Drawdown: {result.metrics.max_drawdown:.2%}")
    print(f"Num Trades: {result.metrics.num_trades}")

In [ ]:
# Visualization
if result.snapshots:
    import matplotlib.pyplot as plt
    
    equity = [s.equity for s in result.snapshots]
    timestamps = [s.timestamp for s in result.snapshots]
    
    fig, axes = plt.subplots(2, 1, figsize=(12, 8))
    
    # Equity curve
    axes[0].plot(timestamps, equity)
    axes[0].set_title('Equity Curve')
    axes[0].set_ylabel('Equity ($)')
    axes[0].axhline(y=10000, color='gray', linestyle='--', alpha=0.5)
    axes[0].grid(True)
    
    # Drawdown
    equity_series = pd.Series(equity, index=timestamps)
    rolling_max = equity_series.expanding().max()
    drawdown = (equity_series - rolling_max) / rolling_max
    
    axes[1].fill_between(timestamps, drawdown, 0, alpha=0.5, color='red')
    axes[1].set_title('Drawdown')
    axes[1].set_ylabel('Drawdown (%)')
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.show()

## 6. Futures Example (Long/Short)

Futures allow short positions → `demean()` can be used

In [ ]:
class MomentumFuturesStrategy(Strategy):
    """Momentum Strategy (Futures Long/Short)."""
    
    name = "Momentum_Futures"
    
    data = {
        "ohlcv": OHLCVSpec(
            exchange="gateio",
            market_type="spot",
            timeframe="1d",
        ),
    }
    
    universe = CryptoLiquid(top_n=30, min_dollar_volume=100_000)
    rebalance_freq = "1d"
    
    def warmup_periods(self) -> int:
        return 30
    
    def compute_signal(self):
        close = self.provider["close"]
        
        returns_20d = operator.ts_returns(close, period=20)
        signal = operator.rank(returns_20d)
        signal = operator.demean(signal)  # Futures: long/short!
        signal = operator.l1_norm(signal)
        
        return signal

# Backtest
futures_result = engine.run(
    strategy=MomentumFuturesStrategy(),
    mode="backtest",
    start=START,
    end=END,
    initial_capital=10000.0,
    market_type="futures",
    leverage=2.0,
    verbose=True,
)

print(f"\n=== Futures Results ===")
if futures_result.metrics:
    print(f"Total Return: {futures_result.metrics.total_return:.2%}")
    print(f"Sharpe Ratio: {futures_result.metrics.sharpe_ratio:.2f}")
    print(f"Max Drawdown: {futures_result.metrics.max_drawdown:.2%}")

## Summary

### Key Concepts

| Component | Description |
|-----------|-------------|
| `OHLCVSpec` | Specify exchange, market_type, timeframe |
| `CryptoLiquid` | Dynamic Universe - filter by volume |
| `DataProvider` | Data loading and access |
| `operator.*` | Pure operator-based operations |
| `Strategy` | Strategy definition (implement compute_signal) |
| `Engine` | Run backtest |

### Spot vs Futures

| | Spot | Futures |
|---|---|---|
| Weight | [0, 1] positive only | [-1, 1] |
| Transform | `rank → l1_norm` | `rank → demean → l1_norm` |
| Leverage | 1x | 1x ~ 125x |

In [ ]:
print("Quickstart complete!")